# Quick Start with Chora

This notebook introduces the core concepts of Chora - a platial modelling library for representing human experience of place.

## Core Concepts

- **Agent**: A person, animal, or entity that experiences place
- **SpatialExtent**: A geographic location or area
- **Encounter**: A moment when an agent is present at a place
- **Familiarity**: How well an agent knows a place (built through repeated encounters)
- **Affect**: The emotional qualities associated with place experiences

In [ ]:
# Install chora if not already installed
# !pip install chora

## 1. Creating a Graph

In [ ]:
from chora.core import PlatialGraph, Agent, SpatialExtent, Encounter, PlatialEdge
from datetime import datetime, timedelta

# Create a new graph to hold our platial model
graph = PlatialGraph(name="my_first_graph")
print(f"Created graph: {graph.name}")

## 2. Adding Agents and Places

In [ ]:
# Create an agent (a person)
alice = Agent.individual("alice")
graph.add_node(alice)

# Create some places
home = SpatialExtent.from_point(lon=-0.09, lat=51.51, name="Home")
park = SpatialExtent.from_point(lon=-0.10, lat=51.52, name="Victoria Park")
cafe = SpatialExtent.from_point(lon=-0.095, lat=51.515, name="Corner Cafe")

graph.add_node(home)
graph.add_node(park)
graph.add_node(cafe)

print(f"Graph now has {graph.node_count} nodes")

## 3. Recording Encounters

In [ ]:
# Record some encounters at the park over several days
encounters = []
for i in range(5):
    enc = Encounter(
        agent_id=alice.id,
        extent_id=park.id,
        start_time=datetime.now() - timedelta(days=i*3),
        activity="walking"
    )
    graph.add_node(enc)
    
    # Create edges connecting agent -> encounter -> place
    graph.add_edge(PlatialEdge.participates_in(alice.id, enc.id))
    graph.add_edge(PlatialEdge.occurs_at(enc.id, park.id))
    encounters.append(enc)

print(f"Created {len(encounters)} encounters at the park")

## 4. Deriving Familiarity

In [ ]:
from chora.derive import update_familiarity

# Update familiarity based on each encounter
for enc in encounters:
    update_familiarity(graph, enc)

# The park should now have higher familiarity for Alice
print("Familiarity updated based on encounters!")

## 5. Querying the Graph

In [ ]:
from chora.query import find_familiar_places

# Find places where Alice has developed familiarity
familiar_places = find_familiar_places(graph, alice.id, min_familiarity=0.1)

print(f"Alice is familiar with {len(familiar_places)} places:")
for place in familiar_places:
    print(f"  - {place.extent.name}: familiarity = {place.familiarity_score:.2f}")

## 6. Visualizing the Graph

In [ ]:
# Export to D3.js format for visualization
import json

nodes = []
links = []

for node in graph.nodes():
    nodes.append({
        "id": str(node.id),
        "type": node.node_type.name,
        "name": getattr(node, "name", str(node.id)[:20])
    })

for edge in graph.edges():
    links.append({
        "source": str(edge.source_id),
        "target": str(edge.target_id),
        "type": edge.edge_type.name
    })

d3_data = {"nodes": nodes, "links": links}
print(json.dumps(d3_data, indent=2))

## Next Steps

- **02_gps_to_places.ipynb**: Load real GPS traces and extract places
- **03_practice_detection.ipynb**: Detect routines and habits
- **04_affective_mapping.ipynb**: Add emotional qualities to places

For more information, see the [Chora documentation](https://chora.readthedocs.io).